In [1]:
%matplotlib inline
import sys 
import os
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import scipy.stats as st
import pandas as pd
import random
from itertools import combinations
import time
import seaborn as sns

In [2]:
def initialize_PPI():

    PPI = nx.Graph()
    PPI.add_edges_from(PPIdata[:, 2:4])
    PPI.remove_edges_from(PPI.selfloop_edges()) #remove self-loops
    PPI_LCC = max(list(nx.connected_component_subgraphs(PPI)), key=len)
    
    return PPI_LCC

In [3]:
def initialize_PPI_nolit():

    PPI = nx.Graph()
    PPI.add_edges_from(PPIdata1[PPIdata1['sources']!='CCSB_LCI'][['gene_symbol_1', 'gene_symbol_2']].values)
    PPI.remove_edges_from(PPI.selfloop_edges()) #remove self-loops
    PPI_LCC = max(list(nx.connected_component_subgraphs(PPI)), key=len)
    
    return PPI_LCC

In [4]:
def initialize_PPI_weighted():

    PPI = nx.Graph()
    PPI.add_edges_from(PPIdata[:, 2:4], weight=1)
    PPI.remove_edges_from(PPI.selfloop_edges()) #remove self-loops
    PPI_LCC = max(list(nx.connected_component_subgraphs(PPI)), key=len)
    
    return PPI_LCC

In [5]:
def initialize_PPI_nolitcorum():

    PPI = nx.Graph()
    PPI.add_edges_from(PPIdata1[~((PPIdata1['sources']=='CCSB_LCI') | (PPIdata1['sources']=='old_PPI_CORUM'))]\
                       [['gene_symbol_1', 'gene_symbol_2']].values)
    PPI.remove_edges_from(PPI.selfloop_edges()) #remove self-loops
    PPI_LCC = max(list(nx.connected_component_subgraphs(PPI)), key=len)
    
    return PPI_LCC

In [6]:
def initialize_PPI_nolitcorum_entrez():

    PPI = nx.Graph()
    PPI.add_edges_from(PPIdata1[~((PPIdata1['sources']=='CCSB_LCI') | (PPIdata1['sources']=='old_PPI_CORUM'))]\
                       [['#gene_ID_1', 'gene_ID_2']].values.astype(str))
    PPI.remove_edges_from(PPI.selfloop_edges()) #remove self-loops
    PPI_LCC = max(list(nx.connected_component_subgraphs(PPI)), key=len)
    
    return PPI_LCC

In [7]:
def initialize_PPI_nolitcorum_weighted():

    PPI = nx.Graph()
    PPI.add_edges_from(PPIdata1[~((PPIdata1['sources']=='CCSB_LCI') | (PPIdata1['sources']=='old_PPI_CORUM'))]\
                       [['gene_symbol_1', 'gene_symbol_2']].values, weight=1)
    PPI.remove_edges_from(PPI.selfloop_edges()) #remove self-loops
    PPI_LCC = max(list(nx.connected_component_subgraphs(PPI)), key=len)
    
    return PPI_LCC

In [8]:
def MT(PPI, seeds, targets, CoA=None):

    if CoA != None:
        PPI.add_edges_from([(e1, e2) for e1, e2 in CoA.edges()])
    
    E_PPI = np.zeros(len(PPI.nodes()))
    for n, gene in enumerate(PPI.nodes()):
        d = []
        for t in seeds:
            d.append(1.0/(nx.shortest_path_length(PPI, source=gene, target=t)+1))
        E_PPI[n] = sum(d)

    sort_PPI = np.array(PPI.nodes())[np.argsort(-E_PPI)]

    TPR_PPI = np.zeros(len(sort_PPI))
    FPR_PPI = np.zeros(len(sort_PPI))
    precision_PPI = np.zeros(len(sort_PPI))

    for n in range(len(sort_PPI)):
        TP = len(set(sort_PPI[0:n+1]) & set(targets))
        FP = len(set(sort_PPI[0:n+1]) - set(targets))
        TN = len(sort_PPI) - len(set(sort_PPI[0:n+1]) | set(targets))
        FN = len(set(targets) - set(sort_PPI[0:n+1]))

        TPR_PPI[n] = 1.0 * TP / (TP+FN)
        FPR_PPI[n] = 1.0 * FP / (FP+TN)
        precision_PPI[n] = 1.0 * TP / (TP+FP)

    PPI_auroc = sum(TPR_PPI)/len(TPR_PPI)
    PPI_percentiles = [np.percentile(E_PPI, i) for i in range(0, 101)]
    target_Es_PPI = [E_PPI[np.where(np.array(PPI.nodes())==i)[0][0]] for i in targets]

    return (PPI_auroc, PPI_percentiles, target_Es_PPI, TPR_PPI, FPR_PPI, precision_PPI, sort_PPI)

In [9]:
def MT_cv(PPI, seeds, targets, kfold, interval_width, CoA=None):
    
    if CoA != None:
        PPI.add_edges_from([(e1, e2) for e1, e2 in CoA.edges()])
    
    cv_sets = [targets[i::kfold] for i in xrange(kfold)]
    intervals = range(0, len(PPI), interval_width)
    auroc_PPI = np.zeros(kfold)
    TPR_PPI = np.zeros((kfold, len(intervals)))
    FPR_PPI = np.zeros((kfold, len(intervals)))
    precision_PPI = np.zeros((kfold, len(intervals)))

    for k in range(kfold):

        E_PPI = np.zeros(len(PPI.nodes()))
        for n, gene in enumerate(PPI.nodes()):
            d = []
            for t in set(seeds) - set(cv_sets[k]):
                d.append(1.0/(nx.shortest_path_length(PPI, source=gene, target=t)+1))
            E_PPI[n] = sum(d)

        sort_PPI = np.array(PPI.nodes())[np.argsort(-E_PPI)]

        for ix, n in enumerate(intervals):
            TP = len(set(sort_PPI[0:n+1]) & set(cv_sets[k]))
            FP = len(set(sort_PPI[0:n+1]) - set(cv_sets[k]))
            TN = len(sort_PPI) - len(set(sort_PPI[0:n+1]) | set(cv_sets[k]))
            FN = len(set(cv_sets[k]) - set(sort_PPI[0:n+1]))

            TPR_PPI[k, ix] = 1.0 * TP / (TP+FN)
            FPR_PPI[k, ix] = 1.0 * FP / (FP+TN)
            precision_PPI[k, ix] = 1.0 * TP / (TP+FP)

        auroc_PPI[k] = sum(TPR_PPI[k, :])/len(TPR_PPI[k, :])

    TPR_PPI_mean = np.mean(TPR_PPI, axis=0)
    FPR_PPI_mean = np.mean(FPR_PPI, axis=0)
    TPR_PPI_std = np.std(TPR_PPI, axis=0)
    TPR_PPI_upper = np.minimum(TPR_PPI_mean + TPR_PPI_std, 1)
    TPR_PPI_lower = TPR_PPI_mean - TPR_PPI_std
    precision_PPI_mean = np.mean(precision_PPI, axis=0)
    auroc_PPI_avg = sum(TPR_PPI_mean)/len(TPR_PPI_mean) 

    return (auroc_PPI_avg, auroc_PPI, TPR_PPI_mean, FPR_PPI_mean, TPR_PPI_std, TPR_PPI_upper, TPR_PPI_lower,\
           precision_PPI_mean)

In [10]:
# input format:(TPR_1, FPR_1, AUROC_1, Label_1, TPR_2, FPR_2, AUROC_2, Label_2, and so on., + outdir='path')
def plot_ROC(*args, **kwargs):

    fig = plt.figure()
    plt.figure(figsize=(8, 6))
    colors = ['#ff00ff', '#ffec00', '#ff3600', '#00b7ff']
    with sns.axes_style("ticks"):
        for i in range(len(args)/4):          
            plt.plot(args[(4*i)+1], args[(4*i)], color=colors[i], \
                                     label='%s, AUC=%0.3f' % (args[(4*i)+3], args[(4*i)+2]))           
        plt.plot([0, 1], [0, 1], 'k--')
        plt.xlabel('FPR')
        plt.ylabel('TPR')
        plt.xlim(-0.05, 1.05)
        plt.ylim(-0.05, 1.05)
        plt.legend(loc=0)
        sns.despine(offset=10, trim=True)
        plt.savefig('%s.pdf' % kwargs['outdir'], format='pdf')
        plt.show()    

In [11]:
# input format:(TPR_1, FPR_1, AUROC_1, Label_1, TPR_2, FPR_2, AUROC_2, Label_2, and so on., + outdir='path')
def plot_ROC_withCoA(*args, **kwargs):

    fig = plt.figure()
    plt.figure(figsize=(8, 6))
    colors = ['#ffec00', '#ff3600', '#00b7ff', '#ff00ff', '#ffec00', '#ff3600', '#00b7ff']
    with sns.axes_style("ticks"):
        for i in range(len(args)/4):          
            if i <= 2:
                plt.plot(args[(4*i)+1], args[(4*i)], color=colors[i], \
                                         label='%s, AUC=%0.3f' % (args[(4*i)+3], args[(4*i)+2]), dashes=[4, 2])
            else:
                plt.plot(args[(4*i)+1], args[(4*i)], color=colors[i], \
                                         label='%s, AUC=%0.3f' % (args[(4*i)+3], args[(4*i)+2]))   
        plt.plot([0, 1], [0, 1], 'k--')
        plt.xlabel('FPR')
        plt.ylabel('TPR')
        plt.xlim(-0.05, 1.05)
        plt.ylim(-0.05, 1.05)
        plt.legend(loc=0)
        sns.despine(offset=10, trim=True)
        plt.savefig('%s.pdf' % kwargs['outdir'], format='pdf')
        plt.show() 

In [12]:
# input format:(TPR_1, FPR_1, lower_TPR1, upper_TPR1, AUROC_1, Label_1, TPR_2, FPR_2, lower_TPR2, 
# upper_TPR2, AUROC_2, Label_2, and so on., + outdir='path')
def plot_ROC_cv_shaded(*args, **kwargs):
    
    sns.set(font_scale=1.5)
    fig = plt.figure()
    fig.set_size_inches(8, 6)
    colors = ['#ffec00', '#ff3600', '#00b7ff', '#ff00ff', '#ffec00', '#ff3600', '#00b7ff']
    
    with sns.axes_style("ticks"):
        for i in range(len(args)/6):         
            plt.plot(args[(6*i)+1], args[(6*i)], color=colors[i], label='%s, AUC=%0.3f' % (args[(6*i)+5], args[(6*i)+4]))
            plt.fill_between(args[(6*i)+1], args[(6*i)+2], args[(6*i)+3], color=colors[i], alpha=0.1)

        plt.plot([0, 1], [0, 1], 'k--')
        plt.xlim(-0.05, 1.05)
        plt.ylim(-0.05, 1.05)
        plt.legend(loc=0)
        sns.despine(offset=10, trim=True)
        plt.xlabel('FPR')
        plt.ylabel('TPR')  
        plt.savefig('%s.pdf' % kwargs['outdir'], format='pdf')
        plt.show()

In [13]:
# input format:(TPR_1, FPR_1, lower_TPR1, upper_TPR1, AUROC_1, Label_1, TPR_2, FPR_2, lower_TPR2, 
# upper_TPR2, AUROC_2, Label_2, and so on., + outdir='path')
def plot_ROC_cv_shaded_separate(*args, **kwargs):
    
    sns.set(font_scale=1.5)
    fig = plt.figure()
    fig.set_size_inches(20, 10)
    colors = ['#ffec00', '#ff3600', '#00b7ff', '#ff00ff', '#ffec00', '#ff3600', '#00b7ff']
    
    with sns.axes_style("ticks"):
        for i in range(len(args)/6):         
            plt.subplot(2, 4, i+1)
            if i <= 2:
                plt.plot(args[(6*i)+1], args[(6*i)], color=colors[i], label='%s, AUC=%0.3f' % \
                         (args[(6*i)+5], args[(6*i)+4]), dashes=[4, 2])
            else:
                plt.plot(args[(6*i)+1], args[(6*i)], color=colors[i], label='%s, AUC=%0.3f' % \
                         (args[(6*i)+5], args[(6*i)+4]))
            plt.fill_between(args[(6*i)+1], args[(6*i)+2], args[(6*i)+3], color=colors[i], alpha=0.1)
            plt.plot([0, 1], [0, 1], 'k--')
            plt.xlim(-0.05, 1.05)
            plt.ylim(-0.05, 1.05)
            plt.legend(loc=0)
            sns.despine(offset=10, trim=True)
            plt.xlabel('FPR')
            plt.ylabel('TPR')  
        plt.savefig('%s.pdf' % kwargs['outdir'], format='pdf')
        plt.show()

In [14]:
# input format:(TPR_1, Precision_1, Label_1, TPR_2, FPR_2, Label_2, and so on., + outdir='path')
def plot_PR(*args, **kwargs):

    fig = plt.figure()
    plt.figure(figsize=(8, 6))
    colors = ['#ff00ff', '#ffec00', '#ff3600', '#00b7ff']
    with sns.axes_style("ticks"):
        for i in range(len(args)/3):          
            plt.plot(args[(3*i)], args[(3*i)+1], color=colors[i], label='%s' % (args[(3*i)+2]))           
        plt.xlabel('Recall')
        plt.ylabel('Precision')
        plt.legend(loc=0)
        sns.despine(offset=10, trim=True)
        plt.savefig('%s.pdf' % kwargs['outdir'], format='pdf')
        plt.show()  

In [15]:
# input format:(targ_Es_1, perc_1, targ_Es_2, perc_2, and so on., + outdir='path')
def plot_violin(*args, **kwargs):

    plt.figure(figsize=(8, 8))
    colors = ['#ff00ff', '#ffec00', '#ff3600', '#00b7ff']
    with sns.axes_style("ticks"):        
        for i in range(len(args)/2):          
            plt.subplot(4, 1, i+1)
            if i == 0: plt.title('Prediction Percentile Rank')
            sns.violinplot(np.digitize(args[(2*i)], args[(2*i)+1]), inner='quartile', color=colors[i])        
            plt.xlim(0, 100)
            plt.xticks(np.arange(0, 110, 10), [('%i%%' % i) for i in np.arange(0, 110, 10)])
        plt.tight_layout()
        plt.savefig('%s.pdf' % kwargs['outdir'], format='pdf')
        plt.show()  

In [16]:
# Given a graph G and (optionally) bins, outputs the degree and binned degree dictionary.
def degree_log_bin(G, nodes=None, log_bins=None):
    
    if nodes is None:
        deg_dict = dict(G.degree())
    else:
        deg_dict = dict(nx.degree(G, nodes))
    #print deg_dict
    
    if log_bins is None:
        log_bins = np.unique(np.logspace(0, np.ceil(np.log10(max(deg_dict.values()))), num=40).astype(int))
    #print log_bins
    
    deg_arr = np.zeros((len(deg_dict), 2))
    for n, entrez in enumerate(sorted([int(x) for x in deg_dict.keys()])):
        deg_arr[n, 0] = entrez
        deg_arr[n, 1] = deg_dict[str(entrez)]

    bin_idx = np.digitize(deg_arr[:, 1], log_bins, right=True)

    # empty bins
    # print set(np.arange(len(log_bins))) - set(bin_idx)

    deg_bin_dict = {}
    for i in set(bin_idx):
        deg_bin_dict[i] = deg_arr[np.where(bin_idx==i)[0], 0].astype(int).astype(str)   
    
    return deg_dict, deg_bin_dict

In [17]:
def select_deg_log_bin(deg_bin_dict, target_deg, log_bins):
    
    if target_deg <= log_bins[np.abs(log_bins - target_deg).argmin()]:
        target_bin_idx = np.abs(log_bins - target_deg).argmin()
    else:
        target_bin_idx = np.abs(log_bins - target_deg).argmin() + 1
    #target_bin = log_bins[target_bin_idx]
    
    return deg_bin_dict[target_bin_idx]

In [18]:
def deg_pres_edge_add(PPI, CoA, bins_dict, bin_edges, deg_dict):
    
    rand_edges = []
    
    for e1, e2 in CoA.edges():
        
        if (e1 in PPI) & (e2 in PPI):
            randset_e1 = select_deg_log_bin(bins_dict, deg_dict[e1], bin_edges)
            randset_e2 = select_deg_log_bin(bins_dict, deg_dict[e2], bin_edges)            
        elif e1 in PPI == False:
            randset_e1 = select_deg_log_bin(bins_dict, 0, bin_edges)
            randset_e2 = select_deg_log_bin(bins_dict, deg_dict[e2], bin_edges) 
        elif e2 in PPI == False:
            randset_e1 = select_deg_log_bin(bins_dict, deg_dict[e1], bin_edges)
            randset_e2 = select_deg_log_bin(bins_dict, 0, bin_edges)             

        e1_rand = random.sample(randset_e1, 1)[0]
        e2_rand = random.sample(randset_e2, 1)[0]
        
        if (PPI.has_edge(e1_rand, e2_rand) == False) & (e1_rand != e2_rand):
            rand_edges.append((e1_rand, e2_rand))            

#         (e1_rand, e2_rand) = list(PPI.edges())[0] # set random edge to a PPI edge to start the while loop
#         while PPI.has_edge(e1_rand, e2_rand) == True:
#             e1_rand = random.sample(randset_e1, 1)[0]
#             e2_rand = random.sample(randset_e2, 1)[0]
#         rand_edges.append((e1_rand, e2_rand))
        
    return rand_edges

In [19]:
def MT_rand(seeds, targets, CoA, bins_dict, bin_edges, deg_dict, Nav):
    
    PPI_auroc = np.zeros(Nav)
    
    for i in range(Nav):
        
        PPI = initialize_PPI_nolitcorum() # _nolitcorum also     
        CoA_rand_edges = deg_pres_edge_add(PPI, CoA, bins_dict, bin_edges, deg_dict)
        print len(CoA_rand_edges),
        PPI.add_edges_from([(e1, e2) for e1, e2 in CoA_rand_edges])

        E_PPI = np.zeros(len(PPI.nodes()))
        for n, gene in enumerate(PPI.nodes()):
            d = []
            for t in seeds:
                d.append(1.0/(nx.shortest_path_length(PPI, source=gene, target=t)+1))
            E_PPI[n] = sum(d)

        sort_PPI = np.array(PPI.nodes())[np.argsort(-E_PPI)]

        TPR_PPI = np.zeros(len(sort_PPI))
        FPR_PPI = np.zeros(len(sort_PPI))
        precision_PPI = np.zeros(len(sort_PPI))

        for n in range(len(sort_PPI)):
            TP = len(set(sort_PPI[0:n+1]) & set(targets))
            FP = len(set(sort_PPI[0:n+1]) - set(targets))
            TN = len(sort_PPI) - len(set(sort_PPI[0:n+1]) | set(targets))
            FN = len(set(targets) - set(sort_PPI[0:n+1]))

            TPR_PPI[n] = 1.0 * TP / (TP+FN)
            FPR_PPI[n] = 1.0 * FP / (FP+TN)
            precision_PPI[n] = 1.0 * TP / (TP+FP)

        PPI_auroc[i] = sum(TPR_PPI)/len(TPR_PPI)

    return PPI_auroc

In [20]:
def MT_optim(PPI, seeds, targets, CoA=None, CoA_weight=None):

    if (CoA != None) & (CoA_weight!=None):
        PPI.add_edges_from([(e1, e2) for e1, e2 in CoA.edges()], weight=CoA_weight)
   
    E_PPI = np.zeros(len(PPI.nodes()))
    for n, gene in enumerate(PPI.nodes()):
        d = []
        for t in seeds:
            path = nx.shortest_path(PPI, source=gene, target=t)
            wts = []
            for ix in range(len(path)-1):
                wts.append(PPI[path[ix]][path[ix+1]]['weight'])
            d.append(1.0/(sum(wts)+1))
        E_PPI[n] = sum(d)

    sort_PPI = np.array(PPI.nodes())[np.argsort(-E_PPI)]

    TPR_PPI = np.zeros(len(sort_PPI))
    FPR_PPI = np.zeros(len(sort_PPI))
    precision_PPI = np.zeros(len(sort_PPI))

    for n in range(len(sort_PPI)):
        TP = len(set(sort_PPI[0:n+1]) & set(targets))
        FP = len(set(sort_PPI[0:n+1]) - set(targets))
        TN = len(sort_PPI) - len(set(sort_PPI[0:n+1]) | set(targets))
        FN = len(set(targets) - set(sort_PPI[0:n+1]))

        TPR_PPI[n] = 1.0 * TP / (TP+FN)
        FPR_PPI[n] = 1.0 * FP / (FP+TN)
        precision_PPI[n] = 1.0 * TP / (TP+FP)

    PPI_auroc = sum(TPR_PPI)/len(TPR_PPI)
    PPI_percentiles = [np.percentile(E_PPI, i) for i in range(0, 101)]
    target_Es_PPI = [E_PPI[np.where(np.array(PPI.nodes())==i)[0][0]] for i in targets]

    return (PPI_auroc, PPI_percentiles, target_Es_PPI, TPR_PPI, FPR_PPI, precision_PPI, sort_PPI)

In [21]:
def MT_optim_cv(PPI, seeds, targets, kfold, interval_width, CoA=None, CoA_weight=None):
    
    if (CoA != None) & (CoA_weight!=None):
        PPI.add_edges_from([(e1, e2) for e1, e2 in CoA.edges()], weight=CoA_weight)
    
    cv_sets = [targets[i::kfold] for i in xrange(kfold)]
    intervals = range(0, len(PPI), interval_width)
    auroc_PPI = np.zeros(kfold)
    TPR_PPI = np.zeros((kfold, len(intervals)))
    FPR_PPI = np.zeros((kfold, len(intervals)))
    precision_PPI = np.zeros((kfold, len(intervals)))

    for k in range(kfold):

        E_PPI = np.zeros(len(PPI.nodes()))
        for n, gene in enumerate(PPI.nodes()):
            d = []
            for t in set(seeds) - set(cv_sets[k]):
                path = nx.shortest_path(PPI, source=gene, target=t)
                wts = []
                for ix in range(len(path)-1):
                    wts.append(PPI[path[ix]][path[ix+1]]['weight'])
                d.append(1.0/(sum(wts)+1))
            E_PPI[n] = sum(d)

        sort_PPI = np.array(PPI.nodes())[np.argsort(-E_PPI)]

        for ix, n in enumerate(intervals):
            TP = len(set(sort_PPI[0:n+1]) & set(cv_sets[k]))
            FP = len(set(sort_PPI[0:n+1]) - set(cv_sets[k]))
            TN = len(sort_PPI) - len(set(sort_PPI[0:n+1]) | set(cv_sets[k]))
            FN = len(set(cv_sets[k]) - set(sort_PPI[0:n+1]))

            TPR_PPI[k, ix] = 1.0 * TP / (TP+FN)
            FPR_PPI[k, ix] = 1.0 * FP / (FP+TN)
            precision_PPI[k, ix] = 1.0 * TP / (TP+FP)

        auroc_PPI[k] = sum(TPR_PPI[k, :])/len(TPR_PPI[k, :])

    TPR_PPI_mean = np.mean(TPR_PPI, axis=0)
    FPR_PPI_mean = np.mean(FPR_PPI, axis=0)
    TPR_PPI_std = np.std(TPR_PPI, axis=0)
    TPR_PPI_upper = np.minimum(TPR_PPI_mean + TPR_PPI_std, 1)
    TPR_PPI_lower = TPR_PPI_mean - TPR_PPI_std
    precision_PPI_mean = np.mean(precision_PPI, axis=0)
    auroc_PPI_avg = sum(TPR_PPI_mean)/len(TPR_PPI_mean) 

    return (auroc_PPI_avg, auroc_PPI, TPR_PPI_mean, FPR_PPI_mean, TPR_PPI_std, TPR_PPI_upper, TPR_PPI_lower,\
           precision_PPI_mean)

In [22]:
def comb_rank(stim_type, TPR, FPR, pr_ranked, exp_data, abun_data, baseline_abun_data, expTopN, abunTopN):
    
    maxN = np.argmax(TPR + (1-FPR))

    all_top = []
    for t in range(5):
        abun_t = np.array([[gene, abun_data[gene][t]] for gene in abun_data.keys()])
        all_top.extend(set(exp_data.sort_values('%s FC' % stim_type, ascending=0)['Gene Symbol'].values[0:expTopN]) \
                    & set(pr_ranked[0:maxN]) & set(abun_t[np.argsort(-abun_t[:, 1].astype(float)), 0][0:abunTopN]) \
                      & set(baseline_abun_data.keys()))
    
    
    prior_dict = {}
    for gene in set(all_top):
        prior_dict[gene] = np.where(pr_ranked==gene)[0][0]
    prior_ranked_list = np.array(sorted(prior_dict, key=prior_dict.get))
       
    expr_dict = {}
    for gene in set(all_top):
        expr_dict[gene] = np.where(mphage_exp.sort_values('%s FC' % stim_type, ascending=0)['Gene Symbol']\
                                   .values==gene)[0][0]
    expr_ranked_list = np.array(sorted(expr_dict, key=expr_dict.get))    
    
    abun_diff = {}
    for gene in set(all_top):
        abun_diff[gene] = sum(abun_data[gene] - baseline_abun_data[gene])
    abun_ranked_list = np.array(sorted(abun_diff, key=abun_diff.get, reverse=True))   
    
    
    comb_rank = {}
    for gene in set(all_top):
        comb_rank[gene] = (np.where(prior_ranked_list==gene)[0][0] + np.where(expr_ranked_list==gene)[0][0] + \
        np.where(abun_ranked_list==gene)[0][0])/3.0
    comb_rank_sorted = sorted(comb_rank, key=comb_rank.get)    
    
    
    return prior_ranked_list, expr_ranked_list, abun_ranked_list, comb_rank_sorted    

In [23]:
def RWR(sort_PPI, targets):

    TPR_PPI = np.zeros(len(sort_PPI))
    FPR_PPI = np.zeros(len(sort_PPI))
    precision_PPI = np.zeros(len(sort_PPI))

    for n in range(len(sort_PPI)):
        TP = len(set(sort_PPI[0:n+1]) & set(targets))
        FP = len(set(sort_PPI[0:n+1]) - set(targets))
        TN = len(sort_PPI) - len(set(sort_PPI[0:n+1]) | set(targets))
        FN = len(set(targets) - set(sort_PPI[0:n+1]))

        TPR_PPI[n] = 1.0 * TP / (TP+FN)
        FPR_PPI[n] = 1.0 * FP / (FP+TN)
        precision_PPI[n] = 1.0 * TP / (TP+FP)

    PPI_auroc = sum(TPR_PPI)/len(TPR_PPI)

    return (PPI_auroc, TPR_PPI, FPR_PPI, precision_PPI)

In [24]:
def kernel(PPI, seeds, targets, CoA=None):

    if CoA != None:
        PPI.add_edges_from([(e1, e2) for e1, e2 in CoA.edges()])
    
    K_PPI = np.zeros(len(PPI.nodes()))
    for n, gene in enumerate(PPI.nodes()):
        d = []
        for t in seeds:
            d.append(np.exp(-(1.0*nx.shortest_path_length(PPI, source=gene, target=t)+1)/len(seeds)))
        K_PPI[n] = -np.log(sum(d))

    sort_PPI = np.array(PPI.nodes())[np.argsort(K_PPI)]

    TPR_PPI = np.zeros(len(sort_PPI))
    FPR_PPI = np.zeros(len(sort_PPI))
    precision_PPI = np.zeros(len(sort_PPI))

    for n in range(len(sort_PPI)):
        TP = len(set(sort_PPI[0:n+1]) & set(targets))
        FP = len(set(sort_PPI[0:n+1]) - set(targets))
        TN = len(sort_PPI) - len(set(sort_PPI[0:n+1]) | set(targets))
        FN = len(set(targets) - set(sort_PPI[0:n+1]))

        TPR_PPI[n] = 1.0 * TP / (TP+FN)
        FPR_PPI[n] = 1.0 * FP / (FP+TN)
        precision_PPI[n] = 1.0 * TP / (TP+FP)

    PPI_auroc = sum(TPR_PPI)/len(TPR_PPI)
    PPI_percentiles = [np.percentile(K_PPI, i) for i in range(0, 101)]
    target_Ks_PPI = [K_PPI[np.where(np.array(PPI.nodes())==i)[0][0]] for i in targets]

    return (PPI_auroc, PPI_percentiles, target_Ks_PPI, TPR_PPI, FPR_PPI, precision_PPI, sort_PPI)

In [25]:
def MT_neg(PPI, seeds, targets, CoA_pos=None, CoA_neg=None):

    if CoA_pos != None:
        PPI.add_edges_from([(e1, e2) for e1, e2 in CoA_pos.edges()])
    if CoA_neg != None:
        PPI.remove_edges_from([(e1, e2) for e1, e2 in CoA_neg.edges()])
    
    E_PPI = np.zeros(len(PPI.nodes()))
    for n, gene in enumerate(PPI.nodes()):
        d = []
        for t in seeds:
            d.append(1.0/(nx.shortest_path_length(PPI, source=gene, target=t)+1))
        E_PPI[n] = sum(d)

    sort_PPI = np.array(PPI.nodes())[np.argsort(-E_PPI)]

    TPR_PPI = np.zeros(len(sort_PPI))
    FPR_PPI = np.zeros(len(sort_PPI))
    precision_PPI = np.zeros(len(sort_PPI))

    for n in range(len(sort_PPI)):
        TP = len(set(sort_PPI[0:n+1]) & set(targets))
        FP = len(set(sort_PPI[0:n+1]) - set(targets))
        TN = len(sort_PPI) - len(set(sort_PPI[0:n+1]) | set(targets))
        FN = len(set(targets) - set(sort_PPI[0:n+1]))

        TPR_PPI[n] = 1.0 * TP / (TP+FN)
        FPR_PPI[n] = 1.0 * FP / (FP+TN)
        precision_PPI[n] = 1.0 * TP / (TP+FP)

    PPI_auroc = sum(TPR_PPI)/len(TPR_PPI)
    PPI_percentiles = [np.percentile(E_PPI, i) for i in range(0, 101)]
    target_Es_PPI = [E_PPI[np.where(np.array(PPI.nodes())==i)[0][0]] for i in targets]

    return (PPI_auroc, PPI_percentiles, target_Es_PPI, TPR_PPI, FPR_PPI, precision_PPI, sort_PPI)